In [ ]:
import pandas as pd # Python数据分析始于pandas
import datashader as ds
import datashader.transfer_functions as tf
import datashader.reductions

from bokeh.plotting import figure, output_notebook, show
from datashader.utils import lnglat_to_meters # 将经纬度向 Web墨卡托投影
from datashader.bokeh_ext import InteractiveImage
from functools import partial # 函数工厂函数
from datashader.utils import export_image # 导出图片
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno # 多种颜色，用于测试
from IPython.core.display import HTML, display
import bokeh.palettes as bp # bokeh 的颜色
from bokeh.palettes import Blues8 # 蓝色
from colorcet import fire # 火 系列色
from datashader.colors import viridis, inferno, Elevation, Greys9, Hot, Sets1to3, Set1, Set2, Set3 # 多种颜色，用于测试
output_notebook()

In [ ]:
import xarray

In [ ]:
import dask
dir(dask.array)

In [ ]:
#　读取数据并计时, 5m.csv 是从数据库截取的， 要求第一列经度，第二列纬度，第三列情绪值，TAB隔开
%time df = pd.read_csv('5m.csv', sep='\t', index_col=None, header=None, names=['lon', 'lat', 'val'])
df.tail()

In [ ]:
# 投影转换
df['x'], df['y'] = lnglat_to_meters(df['lon'], df['lat'])
df.tail()

In [ ]:
bound = 20026376.39
bounds = x_range, y_range = ((-bound, bound), (int(-bound*0.5), int(bound*0.5))) # 设定绘图边界
plot_width = 3000
plot_height = int(plot_width*0.5)

In [ ]:
# # NYC = x_range, y_range = ((-8242000,-8210000), (4965000,4990000))
# def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
#     p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
#         x_range=bounds['x_range'], y_range=bounds['y_range'], outline_line_color=None,
#         min_border=0, min_border_left=0, min_border_right=0,
#         min_border_top=0, min_border_bottom=0, **plot_args)
    
#     p.axis.visible = False
#     p.xgrid.grid_line_color = None
#     p.ygrid.grid_line_color = None
#     return p
# options = dict(line_color=None, fill_color='blue', size=5)

In [ ]:
background = "black"
export = partial(export_image, export_path="export", background=background)
cm = partial(colormap_select, reverse=(background=="black"))

In [ ]:
# 世界范围绘图
def create_image(x_range, y_range):
    cvs = ds.Canvas(plot_width, plot_height, x_range, y_range)
    agg = cvs.points(df, 'x', 'y', ds.reductions.mean('val')) # 情绪均值
    return tf.shade(agg,
                    cmap=Hot,
#                     color_key=color_key
                   )
create_image(*bounds)
export(create_image(*bounds), 'test_figure')

In [ ]:
# 美国
plot_width = 2000
plot_height = int(plot_width*0.5)
def create_image(x_range, y_range):
    cvs = ds.Canvas(plot_width, plot_height, x_range, y_range)
    agg = cvs.points(df, 'x', 'y', ds.reductions.count()) # 照片计数
    return tf.spread(tf.shade(agg,
                    cmap=Hot,
#                     color_key=color_key
                   ))
USA           = ((-124.72,  -66.95), (23.55, 50.06))
USA = [list(r) for r in lnglat_to_meters(*USA)]
create_image(*USA)
export(create_image(*USA), 'test_figure')

In [ ]:
NewYorkCity   = (( -74.39,  -73.44), (40.51, 40.91))
def subset_df(df, site):
    return df[(df['lon'] >= site[0][0]) & (df['lon'] <= site[0][1]) & (df['lat'] >= site[1][0]) & (df['lat'] <= site[1][1])] 
subset_df(df, NewYorkCity).describe()

In [ ]:
# 纽约市
plot_width = 1000
plot_height = int(plot_width*0.618)
def create_image(x_range, y_range, df=df):
    cvs = ds.Canvas(plot_width, plot_height, x_range, y_range)
    agg = cvs.points(df, 'x', 'y', ds.reductions.count())
    return tf.shade(agg,
                    cmap=Hot,
                    )

NewYorkCity   = (( -74.39,  -73.44), (40.51, 40.91))
site = [list(r) for r in lnglat_to_meters(*NewYorkCity)]
export(create_image(*site, subset_df(df, NewYorkCity)), 'test_figure')

In [ ]:
sample = df
p = base_plot(background_fill_color=background)
# p.add_tile(STAMEN_TONER_BACKGROUND)
export(create_image(x_range=bounds['x_range'], y_range=bounds['y_range']), 'test_figure')

In [ ]:
# InteractiveImage(p, create_image)